In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

### Reading Data

In [3]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2662,7.0,0.23,0.33,1.0,0.043,46.0,110.0,0.99118,3.04,0.65,10.8,6
619,7.4,0.19,0.30,1.4,0.057,33.0,135.0,0.99300,3.12,0.50,9.6,6
699,7.4,0.31,0.48,14.2,0.042,62.0,204.0,0.99830,3.06,0.59,9.4,5
3677,6.9,0.58,0.58,8.2,0.032,29.0,169.0,0.99275,3.28,0.44,12.2,6
3728,6.2,0.20,0.33,5.4,0.028,21.0,75.0,0.99012,3.36,0.41,13.5,7
3611,6.8,0.14,0.18,1.4,0.047,30.0,90.0,0.99164,3.27,0.54,11.2,6
3547,7.3,0.20,0.29,19.9,0.039,69.0,237.0,1.00037,3.10,0.48,9.2,6
3210,5.8,0.33,0.23,5.0,0.053,29.0,106.0,0.99458,3.13,0.52,9.0,5
4446,4.4,0.46,0.10,2.8,0.024,31.0,111.0,0.98816,3.48,0.34,13.1,6
2950,6.0,0.28,0.25,1.8,0.042,8.0,108.0,0.99290,3.08,0.55,9.0,5


### Setting up mlflow server and defining experiment

In [4]:
remote_server_uri = "http://127.0.0.1:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [5]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2022/11/24 13:41:40 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/336711984254309563', creation_time=1669315300484, experiment_id='336711984254309563', last_update_time=1669315300484, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

### Defining functions to track parameters, metrics and artifacts

In [6]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [7]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772677
Save to: mlruns/336711984254309563/246129a29d5e4790bd72093ad3999644/artifacts


In [8]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093893
  R2: 0.20224631822892103
Save to: mlruns/336711984254309563/ff85dbc76d8040acac267b25fda82301/artifacts


In [9]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118586
  R2: 0.2157063843066196
Save to: mlruns/336711984254309563/bc9ffc994a77483992a37750686b4733/artifacts


### Tagging runs of experiments

In [11]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='mlruns/336711984254309563', creation_time=1669315300484, experiment_id='336711984254309563', last_update_time=1669315300484, lifecycle_stage='active', name='ElasticNet_wine', tags={}>, <Experiment: artifact_location='mlruns/0', creation_time=1669314741881, experiment_id='0', last_update_time=1669314741881, lifecycle_stage='active', name='Default', tags={}>]


In [12]:
# get the run
_run = client.get_run(run_id="ff85dbc76d8040acac267b25fda82301")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6155290394093893,
 'r2': 0.20224631822892103,
 'rmse': 0.7859129997062341}, params={'alpha': '0.2', 'l1_ratio': '0.2'}, tags={'mlflow.log-model.history': '[{"run_id": "ff85dbc76d8040acac267b25fda82301", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-11-24 18:42:00.343687", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.10.8", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.1.3", '
                             '"serialization_format": "cloudpickle", "code": '
                             '

In [13]:
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)